[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=DepositAndWithdraw.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=DepositAndWithdraw.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Deposit and Withdraw

This notebook shows how to deposit a quantity of a specific token into a Mango Account, and then withdraw it.

This involves using Mango's root and node banks. Root banks keep track of tokens deposited across all Mango accounts, and node banks hold the actual tokens themselves. Deposit and withdrawal instructions therefore require the addresses of the root and node banks. This code shows how to access and provide those addresses.

In [ ]:
import decimal
import mango
import time

# Use this function to display balances because it always fetches fresh data
def show_balances(context, wallet, group, sol_token):
    # Find the current SPL token balance
    token_account = mango.TokenAccount.fetch_largest_for_owner_and_token(context, wallet.address, sol_token)
    print("Wrapped SOL in wallet:", token_account.value)

    # Find the SOL-specific details in the account
    accounts = mango.Account.load_all_for_owner(context, wallet.address, group)
    basket_sol = accounts[0].find_basket_token(sol_token)
    print("Wrapped SOL in account", basket_sol.net_value)
    print()
    
# Use our hard-coded devnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = mango.Wallet([67,218,68,118,140,171,228,222,8,29,48,61,255,114,49,226,239,89,151,110,29,136,149,118,97,189,163,8,23,88,246,35,187,241,107,226,47,155,40,162,3,222,98,203,176,230,34,49,45,8,253,77,136,241,34,4,80,227,234,174,103,11,124,146])

# Signers are effectively an empty CombinableInstruction that only carries the keys for
# signing transactions
signers: mango.CombinableInstructions = mango.CombinableInstructions.from_wallet(wallet)

# Create a 'devnet' Context
context = mango.ContextBuilder.build(cluster_name="devnet")

# Mango accounts are per-Group, so we need to load the Group first.
group = mango.Group.load(context)

# Find the wrapped SOL token
sol_token = context.token_lookup.find_by_symbol_or_raise("SOL")

print("Before:")
show_balances(context, wallet, group, sol_token)


print("Depositing 0.5 SOL:")

accounts = mango.Account.load_all_for_owner(context, wallet.address, group)
token_info = group.find_token_info_by_token(sol_token)
root_bank = token_info.root_bank
node_bank = root_bank.pick_node_bank(context)

deposit_value = mango.TokenValue(sol_token, decimal.Decimal("0.5"))
source_token_account = mango.TokenAccount.fetch_largest_for_owner_and_token(context, wallet.address, sol_token)
deposit_token_account = mango.TokenAccount(source_token_account.account_info, source_token_account.version, source_token_account.owner, deposit_value)

deposit = mango.build_deposit_instructions(context, wallet, group, accounts[0], root_bank, node_bank, deposit_token_account)

all_deposit_instructions = signers + deposit
deposit_transaction_ids = all_deposit_instructions.execute(context)
print("Deposit transaction IDs:", deposit_transaction_ids)

print("Sleeping for 10 seconds...\n")
time.sleep(10)

print("After deposit:")
show_balances(context, wallet, group, sol_token)

print("Withdrawing 0.5 SOL:")

withdrawal_value = mango.TokenValue(sol_token, decimal.Decimal("0.5"))
destination_token_account = mango.TokenAccount.fetch_largest_for_owner_and_token(context, wallet.address, sol_token)
withdrawal_token_account = mango.TokenAccount(destination_token_account.account_info, destination_token_account.version, destination_token_account.owner, withdrawal_value)

allow_borrow = False
withdraw = mango.build_withdraw_instructions(context, wallet, group, accounts[0], root_bank, node_bank, withdrawal_token_account, allow_borrow)

all_withdraw_instructions = signers + withdraw
withdrawal_transaction_ids = all_withdraw_instructions.execute(context)
print("Withdrawal transaction IDs:", withdrawal_transaction_ids)

print("Sleeping for 10 seconds...\n")
time.sleep(10)

print("After withdrawal:")
show_balances(context, wallet, group, sol_token)

print("Done.")
